# Recursion Limits

In this notebook let's make some tries defining `remaining_steps` inside a custom `state_schema` and seeing if it works for recursion limit control. 

Then after that try and set explicit control flows. I know how to do it in a `Command[Literal["next_possible_node1", "next_possible_node2"]]` fashion, can I use that? 

> Here are some sources for this topic:
> - [Graph-API: Impose a recursion limit](https://langchain-ai.github.io/langgraph/how-tos/graph-api/?utm_source=chatgpt.com#impose-a-recursion-limit);
> - ['Medium' post about recursion limits](https://medium.com/@pankajchandravanshi/df371792c8b9);
> - [Adding `remaining_steps` fixed a recursion error problem](https://stackoverflow.com/questions/79446089/langgraph-create-react-agent-with-sqltoolkit-issue-sorry-need-more-steps-to-pr?utm_source=chatgpt.com).

### Custom state schema

Let's make a very simple example: a custom state with a counter that we increment by $1$ at every step (basically the step counter).

In [37]:
from typing import Annotated
from langgraph.graph import MessagesState

# reducer
def counter_add(current_value: int, value_to_add: int):
    return current_value + value_to_add

# custom state
class CustomState(MessagesState):
    counter: Annotated[int, counter_add]  
    remaining_steps: int 

Also, let's make another state schema for experiments where we use the `RemainingSteps` marker from `langgraph.managed.is_last_step`. LangGraph should recognize this type and manage it automatically.

In [24]:
from langgraph.managed.is_last_step import RemainingSteps

class RemainingStepsState(MessagesState):
    counter: Annotated[int, counter_add] # = 8 setting defaults here won't work 
    remaining_steps: RemainingSteps # = 5 defaults need to be set in graph's initial state

>**Note:** LangGraph does not support default values defined inside your state class (especially with annotated or reduced fields). Always provide defaults at runtime via an `initial_state` (see below). 
>
> *Don’t assign a default inside the class definition or via annotation—LangGraph will ignore it.*

### Create agent

In [3]:
# helper function for printing

from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

In [4]:
# tools

from langchain_core.messages import ToolMessage
from langchain_core.tools import tool, InjectedToolCallId
from langgraph.prebuilt import InjectedState
from typing_extensions import Annotated
from langgraph.types import Command


@tool
def update_counter(
    state: Annotated[CustomState, InjectedState],
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """Updates the internal counter, incrementing it by 1"""
    return Command(
        update={
            "counter": 1,
            "messages": [ToolMessage(content="Counter updated", tool_call_id=tool_call_id)]
        }
    )


@tool
def check_counter_value(state: Annotated[CustomState, InjectedState],
                        tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """Checks the value of the counter"""

    counter = state.get("counter")
    remaining_steps = state.get("remaining_steps")

    return Command(
        update={
            "messages" : [ToolMessage(content=f"Counter value: {counter}\nRemaining steps: {remaining_steps}, type:{type(remaining_steps)}", tool_call_id=tool_call_id)]
        }
    )


#### Create two agents with the two custom states for experiments

In [34]:
# agent

from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[update_counter, check_counter_value],
    prompt="You are an helpful AI assistant.",
    name="counter_agent",
    state_schema=CustomState
)

agent_RemainingSteps = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[update_counter, check_counter_value],
    prompt="You are an helpful AI assistant.",
    name="counter_agent",
    state_schema=RemainingStepsState
)


### Experiments with `remaining_steps` 

#### Using `RemainingSteps`:

In [15]:
# run the agent
from langchain_core.messages import HumanMessage

initial_state = {
    "messages": [HumanMessage(content="Update your counter 3 times and then check its value")],
}

for chunk in agent_RemainingSteps.stream(initial_state):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: counter_agent
Tool Calls:
  update_counter (call_TvXYuy8hgo7UNejSXza26mZT)
 Call ID: call_TvXYuy8hgo7UNejSXza26mZT
  Args:
  update_counter (call_oPQk9hPU3eYnV2UWfAaYfAbR)
 Call ID: call_oPQk9hPU3eYnV2UWfAaYfAbR
  Args:
  update_counter (call_1Lr2p1Efq9GR7hPU8wPeqTB8)
 Call ID: call_1Lr2p1Efq9GR7hPU8wPeqTB8
  Args:


Update from node tools:


================================= Tool Message =================================
Name: update_counter

Counter updated


Update from node tools:


================================= Tool Message =================================
Name: update_counter

Counter updated


Update from node tools:


================================= Tool Message =================================
Name: update_counter

Counter updated


Update from node agent:


================================== Ai Message ==================================
Name: counter_agent


Without providing defaults in the initial state, LangGraph automatically initializes `remaining_steps` to $25$, since we defined it through `RemainingSteps`. Also the `counter` is initialized to $0$ as that's the default for `int` type in Python.

In [7]:
# trying to initialize RemainingSteps to a custom value

initial_state = {
    "messages": [HumanMessage(content="Update your counter 3 times and then check its value")],
    "counter": 0,
    "remaining_steps": 3    # this doesn't work
}

for chunk in agent_RemainingSteps.stream(initial_state):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: counter_agent
Tool Calls:
  update_counter (call_C0M6KdTU3QtSfQn9d0Dz86Xz)
 Call ID: call_C0M6KdTU3QtSfQn9d0Dz86Xz
  Args:
  update_counter (call_OiXZTzb9UWuWMEouZwMhLhJc)
 Call ID: call_OiXZTzb9UWuWMEouZwMhLhJc
  Args:
  update_counter (call_7VBTa4FCPLTQhYAauTe7jA0k)
 Call ID: call_7VBTa4FCPLTQhYAauTe7jA0k
  Args:


Update from node tools:


================================= Tool Message =================================
Name: update_counter

Counter updated


Update from node tools:


================================= Tool Message =================================
Name: update_counter

Counter updated


Update from node tools:


================================= Tool Message =================================
Name: update_counter

Counter updated


Update from node agent:


================================== Ai Message ==================================
Name: counter_agent


In [8]:
initial_state = {
    "messages": [HumanMessage(content="Update your counter 3 times and then check its value")],
    "counter": 0,
    "remaining_steps": RemainingSteps(3)    # this doesn't work
}

for chunk in agent_RemainingSteps.stream(initial_state):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: counter_agent
Tool Calls:
  update_counter (call_J36NiZJcazSMOWgrW1vCCEXc)
 Call ID: call_J36NiZJcazSMOWgrW1vCCEXc
  Args:
  update_counter (call_oiprySrSPDocZokCsLa8j0IA)
 Call ID: call_oiprySrSPDocZokCsLa8j0IA
  Args:
  update_counter (call_kdVECgWJMEqJEUfQLv9olZnu)
 Call ID: call_kdVECgWJMEqJEUfQLv9olZnu
  Args:


Update from node tools:


================================= Tool Message =================================
Name: update_counter

Counter updated


Update from node tools:


================================= Tool Message =================================
Name: update_counter

Counter updated


Update from node tools:


================================= Tool Message =================================
Name: update_counter

Counter updated


Update from node agent:


================================== Ai Message ==================================
Name: counter_agent


#### Tries *without* the `RemainingSteps` marker 

In [35]:
# run the agent
from langchain_core.messages import HumanMessage

initial_state = {
    "messages": [HumanMessage(content="Update your counter 3 times and then check its value")],
}

for chunk in agent.stream(initial_state):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: counter_agent
Tool Calls:
  update_counter (call_4BbYqqaiSxlIuvtB3u4waFf9)
 Call ID: call_4BbYqqaiSxlIuvtB3u4waFf9
  Args:
  update_counter (call_u3sva49bzIYn07ChlSu0zJPe)
 Call ID: call_u3sva49bzIYn07ChlSu0zJPe
  Args:
  update_counter (call_AUkUztMMC31ny7DcdbQEik5W)
 Call ID: call_AUkUztMMC31ny7DcdbQEik5W
  Args:


Update from node tools:


================================= Tool Message =================================
Name: update_counter

Error: 1 validation error for update_counter
state.remaining_steps
  Field required [type=missing, input_value={'messages': [HumanMessag...g': 0}})], 'counter': 0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
 Please fix your mistakes.


Update from node tools:


================================= Tool Message =================================
Name: update_counter

Error: 1 validation er

KeyboardInterrupt: 

Interestingly, if we do not use RemainingSteps and initialize with an int we **must** pass an initial state for `remaining_steps`. Notice instead that LangGraph is more flexible about other custom variables, so  even if we don't initialize `counter`, it is set to $0$ anyway by default. 

In [38]:
# run the agent
from langchain_core.messages import HumanMessage

initial_state = {
    "messages": [HumanMessage(content="Update your counter 3 times and then check its value")],
    "remaining_steps" : 1
}

for chunk in agent.stream(initial_state):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================

Sorry, need more steps to process this request.




As we can see, passing a value of $1$ (or even $0$) halts the workflow: so LangGraph seems to automatically check the value of `remaining_steps` even if it's not initialized through `RemainingSteps`. But if we do not implement a custom reducer, the graph won't know how to handle it -> it won't be decrementing `remaining_steps`:

In [40]:
# run the agent
from langchain_core.messages import HumanMessage

initial_state = {
    "messages": [HumanMessage(content="Update your counter 5 times and then check its value")],
    "remaining_steps" : 2
}

for chunk in agent.stream(initial_state):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: counter_agent
Tool Calls:
  update_counter (call_QqslxsJLa8z9i3dKjqbhGv6D)
 Call ID: call_QqslxsJLa8z9i3dKjqbhGv6D
  Args:
  update_counter (call_lFrlXWXxcvRcP0F3oq81st83)
 Call ID: call_lFrlXWXxcvRcP0F3oq81st83
  Args:
  update_counter (call_aDKfyMJ0YXn7zUpuysAlk8f7)
 Call ID: call_aDKfyMJ0YXn7zUpuysAlk8f7
  Args:
  update_counter (call_L6Xie0f0SMF0iElhYChEUbfw)
 Call ID: call_L6Xie0f0SMF0iElhYChEUbfw
  Args:
  update_counter (call_KsiHDD2kspXpWNw0uX85E5sH)
 Call ID: call_KsiHDD2kspXpWNw0uX85E5sH
  Args:


Update from node tools:


================================= Tool Message =================================
Name: update_counter

Counter updated


Update from node tools:


================================= Tool Message =================================
Name: update_counter

Counter updated


Update from node tools:


================================= Tool Message =======

So:

* if we initialize it as `RemainingSteps`, LangGraph handles it automatically. BUT we have no control over its default;
* if we initialize it as an `int`:

  * LangGraph checks by default its value to halt the workflow if it's $0$;
  * we can set the default in the `initial_state` and pass it to `.stream`;
  * we need to assign it a reducer that decrements it.


Notice that another way to set the recursion limit is by using graph configuration keys. We can do that passing `config` to `.stream` or when we initialize the agent: 

```python

# agent

from langgraph.prebuilt import create_react_agent

recursion_limit = 3

agent = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[update_counter, check_counter_value],
    prompt="You are an helpful AI assistant.",
    name="counter_agent",
    state_schema=CustomState
).with_config(recursion_limit=recursion_limit)

### Implement a reducer for `remaining_steps: int`

Let's check what happens if we initialize it as an `int` and implement a custom reducer: will the workflow automatically halt when we reach $0$?

We can actually use `counter_add` but tell the tools to update `remaining_steps` with `-1`:

To trigger `END` when we encounter `remaining_steps = 0`, we can return a `Command[Literal[END]]`! That is, in fact, how you create edges using `Command`.

In [ ]:
# tools

from langchain_core.messages import ToolMessage
from langchain_core.tools import tool, InjectedToolCallId
from langgraph.prebuilt import InjectedState
from typing_extensions import Annotated
from langgraph.types import Command


@tool
def update_counter(
    state: Annotated[CustomState, InjectedState],
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """Updates the internal counter, incrementing it by 1"""
    return Command(
        update={
            "counter": 1,
            "remaining_steps" : -1,
            "messages": [ToolMessage(content="Counter updated", tool_call_id=tool_call_id)]
        }
    )


@tool
def check_counter_value(state: Annotated[CustomState, InjectedState],
                        tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """Checks the value of the counter"""

    counter = state.get("counter")
    remaining_steps = state.get("remaining_steps")

    return Command(
        update={
            "messages" : [ToolMessage(content=f"Counter value: {counter}\nRemaining steps: {remaining_steps}, type:{type(remaining_steps)}", tool_call_id=tool_call_id)],
            "remaining_steps" : -1
        }
    )
